INSTALLING THE REQUIRED DEPENDENCIES

In [3]:
!pip install lightly -q

IMPORTING THE REQUIRED DEPENDENCIES

In [4]:
#dependencies for file management and data visualisation
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
import re
from tqdm import tqdm
import pandas as pd
import gc
import psutil
from matplotlib.animation import FuncAnimation
import seaborn as sns
from IPython.display import HTML
import cv2

#pretraining dependencies
from lightly.data import LightlyDataset

#pretraining of SIMCLR MODEL
from lightly.transforms.simclr_transform import SimCLRTransform
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead

#pretraining of BarlowTwins MODEL
from lightly.loss import BarlowTwinsLoss
from lightly.models.modules import BarlowTwinsProjectionHead
from lightly.transforms.byol_transform import (
    BYOLTransform,
    BYOLView1Transform,
    BYOLView2Transform,
)


#fine-tuning the models
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import pytorch_lightning as pl
import torchvision
from torchvision import transforms


from pytorch_lightning import seed_everything

from sklearn.metrics import confusion_matrix, classification_report


seed_value = 42

torch.manual_seed(seed_value)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)
    
seed_everything(seed_value) 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


42

GETTING ALL THE NATURAL IMAGES TO DO THE PRETRAINING ON

In [5]:
imagenet_path='/kaggle/input/tinyimagenet200/tiny-imagenet-200/train'
final_loc='/kaggle/data'
c=0

#os.makedirs(final_loc, exist_ok=True)
os.makedirs(final_loc)

for directory in os.listdir(imagenet_path):
    c+=1
    directory_loc=os.path.join(imagenet_path,directory,'images')
    for file in os.listdir(directory_loc):
        
        OLD_PATH=os.path.join(directory_loc,file)
        NEW_PATH=os.path.join(final_loc,file)
        #os.replace(OLD_PATH,NEW_PATH)
        #print(OLD_PATH, NEW_PATH)
        
        
        shutil.copy(OLD_PATH, NEW_PATH)
        
   
    if(c%20==0):
        print(f'{c} DIRECTORIES TRANSFERED')



20 DIRECTORIES TRANSFERED
40 DIRECTORIES TRANSFERED
60 DIRECTORIES TRANSFERED
80 DIRECTORIES TRANSFERED
100 DIRECTORIES TRANSFERED
120 DIRECTORIES TRANSFERED
140 DIRECTORIES TRANSFERED
160 DIRECTORIES TRANSFERED
180 DIRECTORIES TRANSFERED
200 DIRECTORIES TRANSFERED


**DEFINING THE SIMCLR PRE-TRAINING PIPELINE**

In [6]:
input_size=25
SC_Imagenet_transform = SimCLRTransform(input_size=input_size)

# Create a dataset from your image folder.
SC_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc,
    transform=SC_Imagenet_transform,
)

# Build a PyTorch dataloader.
SC_Imagenet_dataloader = torch.utils.data.DataLoader(
    SC_Imagenet_dataset,                # Pass the dataset to the dataloader.
    batch_size=16,         # A large batch size helps with learning.
    shuffle=True,           # Shuffling is important!
    num_workers=4
)

In [7]:
max_epochs=10

class SimCLR(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        
        self.projection_head = SimCLRProjectionHead(512,512,2048)
        #1st parameter -> input size (from the last layer of the resnet backbone)
        #2nd parameter -> hidden size
        #3rd parameter -> output size

        self.criterion = NTXentLoss()

    def forward(self, x):
        h = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(h)
        return z

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim],[scheduler]
    
torch.cuda.empty_cache()
SimCLR_ImagenetPT_model = SimCLR()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(SimCLR_ImagenetPT_model, SC_Imagenet_dataloader)

Training: 0it [00:00, ?it/s]

In [8]:
SimCLR_ImagenetPT_model.backbone

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

**GOING FOR THE BARLOW TWINS MODEL NOW**

In [9]:
BT_Imagenet_transform = BYOLTransform(
    view_1_transform=BYOLView1Transform(input_size=32, gaussian_blur=0.0),
    view_2_transform=BYOLView2Transform(input_size=32, gaussian_blur=0.0),
)

BT_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc,
    transform=BT_Imagenet_transform,
)

BT_Imagenet_dataloader = torch.utils.data.DataLoader(
    BT_Imagenet_dataset,   
    batch_size=16,         
    shuffle=True,           
    num_workers=4
)

In [10]:
max_epochs=10

class BarlowTwins(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = BarlowTwinsProjectionHead(512, 2048, 2048)
        self.criterion = BarlowTwinsLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(x)
        return z

    def training_step(self, batch, batch_index):
        (x0, x1) = batch[0]
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(self.parameters(), lr=0.06)
        return optim
        
torch.cuda.empty_cache()
BT_ImagenetPT_model = BarlowTwins()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(BT_ImagenetPT_model, BT_Imagenet_dataloader)

Training: 0it [00:00, ?it/s]

**PRETRAINING OVER SWAV MODEL NOW**

In [11]:
from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
from lightly.transforms.swav_transform import SwaVTransform


SW_Imagenet_transform = SwaVTransform()
# we ignore object detection annotations by setting target_transform to return 0
SW_Imagenet_dataset = LightlyDataset(
    input_dir=final_loc,
    transform=SW_Imagenet_transform,
)
# or create a dataset from a folder containing images or videos:
# dataset = LightlyDataset("path/to/folder")

SW_Imagenet_dataloader = torch.utils.data.DataLoader(
    SW_Imagenet_dataset,   
    batch_size=16,         
    shuffle=True,           
    num_workers=4
)


In [13]:
max_epochs=6
class SwaV(pl.LightningModule):
    def __init__(self):
        super().__init__()
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        self.projection_head = SwaVProjectionHead(512, 512, 128)
        self.prototypes = SwaVPrototypes(128, n_prototypes=512)
        self.criterion = SwaVLoss()

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        x = self.projection_head(x)
        x = nn.functional.normalize(x, dim=1, p=2)
        p = self.prototypes(x)
        return p

    def training_step(self, batch, batch_idx):
        self.prototypes.normalize()
        views = batch[0]
        multi_crop_features = [self.forward(view.to(self.device)) for view in views]
        high_resolution = multi_crop_features[:2]
        low_resolution =  multi_crop_features[2:]
        loss = self.criterion(high_resolution, low_resolution)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=0.001)
        return optim

torch.cuda.empty_cache()
SW_ImagenetPT_model = SwaV()
trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
trainer.fit(SW_ImagenetPT_model, SW_Imagenet_dataloader)


Training: 0it [00:00, ?it/s]

**PREPARING THE DATA FOR THE FINE-TUNING OF THESE PRETRAINED MODELS**

In [14]:
def one_hot(n):
    index={0:0,0.5:1,1:2,2:3}
    #print(index)
    #print(index[n])
    y_new=np.zeros(4)
    y_new[index[n]]=1
    #print(y_new)
    return y_new

In [15]:
c=0
df_new = pd.DataFrame(columns=['IMG', 'CDR'])
def make_dataset(path,cdr):
    
    global c, df_new
    
    for i in os.listdir(path):
        c+=1
        image = cv2.imread(os.path.join(path,i))
        image_np = np.array(image)
        image_tensor = transforms.ToTensor()(image_np)
        padded_image = transforms.CenterCrop((224, 224))(transforms.Pad(28)(image_tensor))
        #print(padded_image.shape)
        '''plt.imshow(padded_image.permute(1, 2, 0))
        print('padded_image',padded_image.shape)'''  
        
        new_entry = {
            'IMG': padded_image,
            'CDR': cdr
        } 
        df_new = pd.concat([df_new, pd.DataFrame([new_entry])], ignore_index=True)
        #print(i)


make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented',1)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented',2)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented',0)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented',0.5)
        
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented',1)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented',2)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented',0)
make_dataset('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented',0.5)


In [16]:
print(c,df_new.shape)
print(df_new['IMG'][0].shape)

6400 (6400, 2)
torch.Size([3, 224, 224])


In [17]:
x=np.stack(df_new['IMG'].tolist())
y=df_new['CDR']

y_one_hot=[]
for i in y:
    y_one_hot.append(one_hot(i)) 

In [18]:
print(type(x))

<class 'numpy.ndarray'>


In [19]:
x=torch.tensor(x)
y=torch.Tensor(y_one_hot)

dataset=TensorDataset(x,y)

train_size=int(0.6*len(x))
val_size=int(0.2*len(x))
temp_size=2*val_size

#print(train_size,val_size,temp_size)

train_dataset, temp_dataset = random_split(dataset, [train_size, temp_size])
val_dataset,test_dataset=random_split(temp_dataset, [val_size,val_size])

# Define batch size
#batch_size = 8

# Create a DataLoader for shuffling and batching
#train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)


/tmp/ipykernel_32/701760665.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  y=torch.Tensor(y_one_hot)


**DEFINING FUNCTIONS TO PERFORM THE FINE-TUNING ON DIFFERENT MODELS**

In [20]:
def train_and_val(backbone):
    
    #training the models first on train_dataloader
    
    models=[]
    for i in range(2):
        models.append(ResnetSingleChannel(backbone,4))
        
   
    
    batch_sizes=[64,32]
    
    for model_no in range(2):
        print(f'TRAINING MODEL {model_no+1}')
        print('--------------------------------------------------------------')
        fine_tune_opt = optim.Adam(models[model_no].parameters(), lr=0.001, weight_decay=0.0001)
        ft_loss_fn= nn.CrossEntropyLoss()
        
        batch_size=batch_sizes[model_no]
        
        train_dataloader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)
        

        num_epochs = 5
        total_steps=train_size//batch_size

        for epoch in range(num_epochs):
            i=0

            for x_batch,y_batch in train_dataloader:
                i+=1
                outputs=models[model_no](torch.Tensor(x_batch))
                
                '''print('out',outputs.shape)
                print('y',y_batch.shape)
                print('out',outputs)
                print('y',y_batch)'''
                
                loss = ft_loss_fn(outputs, y_batch)
                fine_tune_opt.zero_grad()
                loss.backward()
                fine_tune_opt.step()

                if (i+1) % 20 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')
    
    # performing validation on validation data_loader
    best_model=-1
    best_loss=np.inf
    for model_no in range(2):
        
        loss_sum=0
        
        for x_batch,y_batch in val_dataloader:
            outputs=models[model_no](torch.Tensor(x_batch))
            loss = ft_loss_fn(outputs, y_batch)
            loss_sum+=loss.item()
            
        if(loss_sum<best_loss):
            best_model=model_no
            best_loss=loss_sum
    
    return models[best_model]
        
        

In [67]:
def metrics(model):
    
    test_dataloader=DataLoader(test_dataset,batch_size=1,shuffle=True)
    
    y_pred_M=[]
    y_true_M=[]
    c=0
    logloss_sum=0
    
    for x_batch,y_batch in test_dataloader:
        
        c+=1
        y_pred=model(x_batch)

        '''y_pred_M.append(np.argmax(y_pred.detach().numpy()))
        y_true_M.append(np.argmax(y_batch.detach().numpy()))'''

        softmax_probs = softmax(y_pred)

        '''print(y_pred)
        print(softmax_probs)
        print(y_batch)'''

        logloss = log_loss(y_batch.detach().numpy(), softmax_probs.detach().numpy())

        logloss_sum+=logloss
        #print(logloss) 

    return(logloss_sum/c)

In [22]:
#MAKING A NEW NETWORK WHICH USES THE BACKBONE OF THE PRETRAINED MODEL, AND HAS AN INPUT LAYER TO TAKE IN A SINGLE CHANNEL IMAGE, AND THEN PASS INTO 
#BACKBONE, AND FINALLY OUTPUT LAYER WHICH PERFORMS REGRESSION

class ResnetSingleChannel(nn.Module):
    def __init__(self,backbone,channels):
        super(ResnetSingleChannel,self).__init__()
        self.input=nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1, bias=False)
        self.backbone=backbone
        self.output=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512, out_features=256),
            nn.Linear(in_features=256, out_features=channels)
        )
        
    def forward(self,x):
        x=self.input(x)
        x=self.backbone(x)
        x=self.output(x)
        
        return x

**FINE TUNING OVER THE SIMCLR MODEL**

In [23]:
for layer in SimCLR_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
SC_MODEL=train_and_val(SimCLR_ImagenetPT_model.backbone)
SC_M_2=metrics(SC_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/5], Step [20/60], Loss: 0.8923
Epoch [1/5], Step [40/60], Loss: 0.7936
Epoch [1/5], Step [60/60], Loss: 0.8855
Epoch [2/5], Step [20/60], Loss: 0.8399
Epoch [2/5], Step [40/60], Loss: 0.7186
Epoch [2/5], Step [60/60], Loss: 0.7142
Epoch [3/5], Step [20/60], Loss: 0.6325
Epoch [3/5], Step [40/60], Loss: 0.5953
Epoch [3/5], Step [60/60], Loss: 0.4949
Epoch [4/5], Step [20/60], Loss: 0.3282
Epoch [4/5], Step [40/60], Loss: 0.4406
Epoch [4/5], Step [60/60], Loss: 0.2990
Epoch [5/5], Step [20/60], Loss: 0.2853
Epoch [5/5], Step [40/60], Loss: 0.1821
Epoch [5/5], Step [60/60], Loss: 0.3797
TRAINING MODEL 2
--------------------------------------------------------------
Epoch [1/5], Step [20/120], Loss: 0.9007
Epoch [1/5], Step [40/120], Loss: 0.8856
Epoch [1/5], Step [60/120], Loss: 0.7238
Epoch [1/5], Step [80/120], Loss: 0.9419
Epoch [1/5], Step [100/120], Loss: 0.7721
Epoch [1/5], Step [120/120], Loss:

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**FINE TUNING OVER THE BARLOW TWINS MODEL**

In [70]:
for layer in BT_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
BT_MODEL=train_and_val(BT_ImagenetPT_model.backbone)
BT_M_2=metrics(BT_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/5], Step [20/60], Loss: 1.7632
Epoch [1/5], Step [40/60], Loss: 1.2376
Epoch [1/5], Step [60/60], Loss: 1.2612
Epoch [2/5], Step [20/60], Loss: 0.9757
Epoch [2/5], Step [40/60], Loss: 0.9629
Epoch [2/5], Step [60/60], Loss: 1.0756
Epoch [3/5], Step [20/60], Loss: 1.0650
Epoch [3/5], Step [40/60], Loss: 0.8951
Epoch [3/5], Step [60/60], Loss: 0.8176
Epoch [4/5], Step [20/60], Loss: 0.8993
Epoch [4/5], Step [40/60], Loss: 1.1381
Epoch [4/5], Step [60/60], Loss: 1.0183
Epoch [5/5], Step [20/60], Loss: 0.9652
Epoch [5/5], Step [40/60], Loss: 0.9370
Epoch [5/5], Step [60/60], Loss: 0.8118
TRAINING MODEL 2
--------------------------------------------------------------
Epoch [1/5], Step [20/120], Loss: 1.3414
Epoch [1/5], Step [40/120], Loss: 2.6125
Epoch [1/5], Step [60/120], Loss: 2.0248
Epoch [1/5], Step [80/120], Loss: 1.2361
Epoch [1/5], Step [100/120], Loss: 0.8813
Epoch [1/5], Step [120/120], Loss:

**FINE TUNING OVER THE SWAV MODEL**

In [71]:
for layer in SW_ImagenetPT_model.backbone:
    for param in layer.parameters():
        param.requires_grad = True
        
SW_MODEL=train_and_val(SW_ImagenetPT_model.backbone)
SW_M_2=metrics(SW_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/5], Step [20/60], Loss: 0.9210
Epoch [1/5], Step [40/60], Loss: 0.9176
Epoch [1/5], Step [60/60], Loss: 0.9756
Epoch [2/5], Step [20/60], Loss: 1.1060
Epoch [2/5], Step [40/60], Loss: 0.9630
Epoch [2/5], Step [60/60], Loss: 0.7968
Epoch [3/5], Step [20/60], Loss: 0.8326
Epoch [3/5], Step [40/60], Loss: 0.8409
Epoch [3/5], Step [60/60], Loss: 0.8075
Epoch [4/5], Step [20/60], Loss: 0.8612
Epoch [4/5], Step [40/60], Loss: 0.8012
Epoch [4/5], Step [60/60], Loss: 0.8534
Epoch [5/5], Step [20/60], Loss: 0.8718
Epoch [5/5], Step [40/60], Loss: 0.8418
Epoch [5/5], Step [60/60], Loss: 0.9419
TRAINING MODEL 2
--------------------------------------------------------------
Epoch [1/5], Step [20/120], Loss: 1.0153
Epoch [1/5], Step [40/120], Loss: 1.0145
Epoch [1/5], Step [60/120], Loss: 0.7883
Epoch [1/5], Step [80/120], Loss: 0.7911
Epoch [1/5], Step [100/120], Loss: 0.8555
Epoch [1/5], Step [120/120], Loss:

**FINE TUNING OVER THE UNTRAINED RESNET MODEL**

In [72]:
resnet = torchvision.models.resnet18()
untrained= nn.Sequential(*list(resnet.children())[:-1])

for layer in untrained:
    for param in layer.parameters():
        param.requires_grad = True

UN_MODEL=train_and_val(untrained)
UN_M_2=metrics(UN_MODEL)

TRAINING MODEL 1
--------------------------------------------------------------
Epoch [1/5], Step [20/60], Loss: 0.9472
Epoch [1/5], Step [40/60], Loss: 0.9067
Epoch [1/5], Step [60/60], Loss: 0.8416
Epoch [2/5], Step [20/60], Loss: 0.8884
Epoch [2/5], Step [40/60], Loss: 0.9664
Epoch [2/5], Step [60/60], Loss: 0.6954
Epoch [3/5], Step [20/60], Loss: 0.7631
Epoch [3/5], Step [40/60], Loss: 0.7042
Epoch [3/5], Step [60/60], Loss: 0.7734
Epoch [4/5], Step [20/60], Loss: 0.5689
Epoch [4/5], Step [40/60], Loss: 0.5364
Epoch [4/5], Step [60/60], Loss: 0.4203
Epoch [5/5], Step [20/60], Loss: 0.3297
Epoch [5/5], Step [40/60], Loss: 0.5171
Epoch [5/5], Step [60/60], Loss: 0.4068
TRAINING MODEL 2
--------------------------------------------------------------
Epoch [1/5], Step [20/120], Loss: 0.2892
Epoch [1/5], Step [40/120], Loss: 0.6161
Epoch [1/5], Step [60/120], Loss: 0.4237
Epoch [1/5], Step [80/120], Loss: 0.4180
Epoch [1/5], Step [100/120], Loss: 0.3245
Epoch [1/5], Step [120/120], Loss:

**PRINTING OUT THE PERFORMANCE METRICS**

In [75]:
print('SimCLR',SC_M_2)
print('Barlow',BT_M_2)
print('Swav',SW_M_2)
print('Untrained',UN_M_2)

SimCLR 1.4556029397360444
Barlow 0.9936796108116001
Swav 1.0323936266081515
Untrained 1.0058041749604016
